 # A/B 테스트 기초
 ## A/B 테스트란?
- 두 가지 버젼을 만들어 어떤 것이 더 나은 성과를 보이는 지 비교하는 데이터 기반 실험 방법
- 과거 A/B 테스트를 이용해 얻은 인사이트
 1. 기능보다 가치 전달에 처점을 맞춘 카피 & 사람들은 되도록 스킵하려고 함
 2. 랜딩 페이지에 심리적 허들을 낮추는 가벼운 카피(Try!) 사용
 3. 랜딩 페이지에 제품 사용을 복잡하게 느껴지는 카피 회피
 4. 랜딩 페이지를 비가입 고객 친화적으로 구성
 5. CTA 버튼의 수 최소화하여 우선순위 높은 기능에 집중
 6. 전체 퍼널의 단계 수를 고려한 디자인
 7. 한 화면에 많지 않은 입력란
 8. 100의 법칙 - 액수가 낮다면 할인가 말고 할인율 노출
 9. 리스트·그리드 UI에 CTA 버튼 추가
 10. 인기 항목은 빠르게 필터링할 수 있도록 디자인
 11. 무한 스크롤은 양날의 검 - 많은 항목을 쉽게 탐색하는 것이 꼭 전환으로 이어지지 않음
 12. 이탈률 감소를 위한 콘텐츠 발행일 숨김

## A/B 테스트의 요소
- A안: 대조군
- B안: 실험군
- 무작위 사용자 그룹
- 핵심성과지표: 어떤 요소를 더 선호하는가? 전환율, 클릭률, 페이지 체류 시간 등

## 왜 필요한가?
1. 직감이 아닌 데이터에 근거
2. 신뢰성과 재현성 높은 결과 확보: 유의미한 결과를 통해 이게 우연이 아니라는 걸 입증
3. 위험 최소화 및 점진적 개선: 베타테스터

## 발전과정 및 사례
- 농업 > 직접 우편 마케팅 > 글로벌 IT 기업
- 사례: 오바마의 6천만달러 버튼, 넷플릭스의 개인화 썸네일, 아마존의 고객 집착, 북킹닷컴의 긴박감 마케팅

## 분석 프레임워크
1. 기획 및 설계
- 가설 수립: 만약 ~한다면 ~라는 결과가 나올 것이다. 왜냐하면 ~이기 때문이다
- 지표 및 성공 기준 정의: 핵심 지표 + 보조 지표
- 샘플 크기 및 실험 기간 산정: 최소 사용자 수를 계산, 샘플 추출을 위한 기간 예측

2. 실행 및 데이터 수집
- 실험 구현 및 실행
- 데이터 수집

3. 분석 및 의사결정
- 결과 분석 및 유의성 검증
- 결론 도출 및 의사결정: 승/패/무
- 결과 공유 및 학습

### 실습!

In [6]:
import pandas as pd

# Load the dataset
df = pd.read_csv(r"C:\Users\Dongchan Kim\pyproject\ab_test.csv")

# 데이터 로드 및 살펴보기
print('---데이터 첫 5행 확인(head)---')
print(df.head())
print('\n'+"="*50 +"\n")

# 2. 기초 통계량 확인
print("---기초 통계량 확인(describe)---")
print(df.describe())
print("\n"+"="*50+"\n")

#3. 그룹별 평균 메트릭 비교
#time을 초 단위로 변환
time_split = df['time'].str.split(':', expand=True)
df['time_sec'] = time_split[0].astype(float) * 60 + time_split[1].astype(float)

#con_treat와 page가 일치하지 않는 데이터 확인 및 제거
#old_page는 control만, new_page는 treatment만
df_cleaned = df[((df['con_treat'] =='control') & (df['page'] == 'old_page')) |
                ((df['con_treat'] == 'treatment') & (df['page'] == 'new_page'))]


#그룹별 평균 계산(전환율, 체류시간)
#converted의 평균이 전환율
mean_metrics = df_cleaned.groupby('con_treat').agg(
    conversion_rate = ('converted', 'mean'),
    time_on_page_sec = ('time_sec', 'mean')
).reset_index()

print("---그룹별 평균 지표 (전환율, 체류 시간)---")
print(mean_metrics)

---데이터 첫 5행 확인(head)---
       id     time  con_treat      page  converted
0  851104  11:48.6    control  old_page          0
1  804228  01:45.2    control  old_page          0
2  661590  55:06.2  treatment  new_page          0
3  853541  28:03.1  treatment  new_page          0
4  864975  52:26.2    control  old_page          1


---기초 통계량 확인(describe)---
                  id      converted
count  294478.000000  294478.000000
mean   787974.124733       0.119659
std     91210.823776       0.324563
min    630000.000000       0.000000
25%    709032.250000       0.000000
50%    787933.500000       0.000000
75%    866911.750000       0.000000
max    945999.000000       1.000000


---그룹별 평균 지표 (전환율, 체류 시간)---
   con_treat  conversion_rate  time_on_page_sec
0    control         0.120386       1804.925762
1  treatment         0.118807       1801.523057


In [11]:
#혹시 아직 있나?
mismatched_df = df_cleaned[((df_cleaned['con_treat']=='control') & (df_cleaned['page'] !='old_page')) |
                   ((df_cleaned['con_treat'] == 'treatment') & (df_cleaned['page'] != 'new_page'))]
mismatched_count = len(mismatched_df)

#중복 데이터 확인(user_id)
duplicate_users = df_cleaned.groupby('id')['con_treat'].nunique()
duplicate_users_count = duplicate_users[duplicate_users > 1].count()

#정합성 맞는 데이터만 필터링
#df_cleaned = df.drop(mismatched_df.index)

#업무 2 KPI 탐구
overall_conversion_rate = df_cleaned['converted'].mean()

#3. 데이터 최종확인
final_info = df_cleaned.info(verbose=False, memory_usage=False)
final_info_string = str(df_cleaned.info()) #info()는 None 반환하므로 문자열로 캡쳐

#최종 데이터셋 그룹별 샘플 수
final_group_counts = df_cleaned['con_treat'].value_counts()

#결과출력
print('업무1 : 데이터 정합성 검증 결과')
print(f'실험 설계 불일치 데이터 수 : {mismatched_count}')
print(f'두 그룹에 모두 속한 중복 사용자 수: {duplicate_users_count}')
print(f'초기 데이터 수: {len(df)}')
print(f'정제 후 최종 데이터 수: {len(df_cleaned)}')

print('\n 업무 2 : 핵심 지표(KPI) 탐구 결과')
print(f'전체 전환율 (Baseline): {overall_conversion_rate}')

print('\n 업무3: 데이터 최종확인 결과')
print(f'최종 데이터 정보 요약:')
df_cleaned.info()
print('\n최종 데이터셋 그룹별 샘플 수:')
print(final_group_counts)

<class 'pandas.core.frame.DataFrame'>
Index: 290585 entries, 0 to 294477
Columns: 6 entries, id to time_sec
dtypes: float64(1), int64(2), object(3)<class 'pandas.core.frame.DataFrame'>
Index: 290585 entries, 0 to 294477
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   id         290585 non-null  int64  
 1   time       290585 non-null  object 
 2   con_treat  290585 non-null  object 
 3   page       290585 non-null  object 
 4   converted  290585 non-null  int64  
 5   time_sec   290585 non-null  float64
dtypes: float64(1), int64(2), object(3)
memory usage: 15.5+ MB
업무1 : 데이터 정합성 검증 결과
실험 설계 불일치 데이터 수 : 0
두 그룹에 모두 속한 중복 사용자 수: 0
초기 데이터 수: 294478
정제 후 최종 데이터 수: 290585

 업무 2 : 핵심 지표(KPI) 탐구 결과
전체 전환율 (Baseline): 0.11959667567149027

 업무3: 데이터 최종확인 결과
최종 데이터 정보 요약:
<class 'pandas.core.frame.DataFrame'>
Index: 290585 entries, 0 to 294477
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype  
---  -----

# 준비 완!

In [13]:
#~~
#KPI 핵심 지표 비교 분석
#그룹별로 전환율과 체류시간의 평균 및 중앙값 계산
group_analysis = df_cleaned.groupby('con_treat').agg(
    user_count = ('id', 'nunique'),
    conversion_rate = ('converted', 'mean'),
    avg_time_on_page = ('time_sec', 'mean'),
    median_time_on_page = ('time_sec', 'median')
).reset_index()

#전환율 차이 및 비율 계산
control_conversion_rate = group_analysis.loc[group_analysis['con_treat'] ==
                                             'control', 'conversion_rate'].iloc[0]
treatment_conversion_rate = group_analysis.loc[group_analysis['con_treat'] ==
                                               'treatment', 'conversion_rate'].iloc[0]

absolute_lift = treatment_conversion_rate -control_conversion_rate
relative_lift_percent = (absolute_lift / control_conversion_rate) * 100

#결과출력
print("그룹별 핵심 지표 비교 분석 결과")
print(group_analysis.to_string())
print('전환율 변화 상세')
print(f'절대적 변화 (Absolute Lift): {absolute_lift:.4f} (Treatment - Control)')
print(f'상대적 변화 (Relative Lift): {relative_lift_percent:.2f}%')



그룹별 핵심 지표 비교 분석 결과
   con_treat  user_count  conversion_rate  avg_time_on_page  median_time_on_page
0    control      145274         0.120386       1804.925762               1803.6
1  treatment      145310         0.118807       1801.523057               1803.6
전환율 변화 상세
절대적 변화 (Absolute Lift): -0.0016 (Treatment - Control)
상대적 변화 (Relative Lift): -1.31%
